# hdfs

In [1]:
!pwd

/data/home/darina.bocharova


In [2]:
!ls

'Create spark session.ipynb'   lab02.ipynb   lab04.ipynb
 features.json		       lab02.json    sber-spark-ds-10
 lab01.ipynb		       lab03.csv    'v_spark scala api.ipynb'
 lab01.json		       lab03.ipynb


In [3]:
!hdfs dfs -ls ./

Found 2 items
drwx------   - darina.bocharova darina.bocharova          0 2022-10-30 03:00 .Trash
drwxr-xr-x   - darina.bocharova darina.bocharova          0 2022-10-26 17:22 .sparkStaging


In [4]:
!hdfs dfs -ls hdfs://spark-master-1.newprolab.com:8020/user/darina.bocharova

Found 2 items
drwx------   - darina.bocharova darina.bocharova          0 2022-10-30 03:00 hdfs://spark-master-1.newprolab.com:8020/user/darina.bocharova/.Trash
drwxr-xr-x   - darina.bocharova darina.bocharova          0 2022-10-26 17:22 hdfs://spark-master-1.newprolab.com:8020/user/darina.bocharova/.sparkStaging


In [5]:
# !hdfs dfs -rm -r hdfs://spark-master-1.newprolab.com:8020/user/darina.bocharova/имя_файла

In [6]:
!hdfs dfs -ls /tmp/darina.bocharova

In [7]:
# !hdfs dfs -rm -r /tmp/darina.bocharova/имя_файла

# SparkSession

In [8]:
import os
import sys

os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 5 --executor-memory 4g --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [9]:
from pyspark.sql import *

spark = SparkSession.builder \
                    .master("local[2]") \
                    .appName("spark-course") \
                    .config("spark.driver.memory", "512m") \
                    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5") \
                    .getOrCreate()

In [10]:
spark

In [11]:
import pyspark.sql.types as t
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType, FloatType
from pyspark.ml.feature import HashingTF, Tokenizer, StringIndexer, IndexToString, IDF
from pyspark.ml.classification import RandomForestClassifier

# Train

In [12]:
!hdfs dfs -ls /labs/slaba04/

Found 1 items
-rw-r--r--   3 hdfs hdfs  655090069 2022-01-06 18:46 /labs/slaba04/gender_age_dataset.txt


In [13]:
!hdfs dfs -head /labs/slaba04/gender_age_dataset.txt

gender	age	uid	user_json
F	18-24	d50192e5-c44e-4ae8-ae7a-7cfe67c8b777	{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}
M	25-34	d502331d-621e-4721-ada2-5d30b2c3801f	{"visits": [{"url": "http://sweetrading.ru/?p=900", "timestamp": 1419717886224}, {"url": "http://sweetrading.ru/?p=884", "timestamp": 1419717884437}, {"url": "http://sweetrading.ru/?

In [14]:
path = '/labs/slaba04/gender_age_dataset.txt'

In [15]:
train_schema = t.StructType(fields=[
    t.StructField("gender", t.StringType()),
    t.StructField("age", t.StringType()),
    t.StructField("uid", t.StringType()),
    t.StructField("user_json", t.StringType())
])

In [16]:
raw_train = spark.read.csv(path, sep="\t", header=True, schema=train_schema)

In [17]:
raw_train.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- user_json: string (nullable = true)



In [18]:
raw_train.show()

+------+-----+--------------------+--------------------+
|gender|  age|                 uid|           user_json|
+------+-----+--------------------+--------------------+
|     F|18-24|d50192e5-c44e-4ae...|{"visits": [{"url...|
|     M|25-34|d502331d-621e-472...|{"visits": [{"url...|
|     F|25-34|d50237ea-747e-48a...|{"visits": [{"url...|
|     F|25-34|d502f29f-d57a-46b...|{"visits": [{"url...|
|     M| >=55|d503c3b2-a0c2-4f4...|{"visits": [{"url...|
|     F|25-34|d5090ddf-5648-487...|{"visits": [{"url...|
|     F|25-34|d50bcef8-16ff-4e8...|{"visits": [{"url...|
|     F|18-24|d50e23dc-0cbd-488...|{"visits": [{"url...|
|     F|45-54|d50fdabb-4208-441...|{"visits": [{"url...|
|     F|18-24|d511b480-23a6-482...|{"visits": [{"url...|
|     F|25-34|d51294ed-1b95-4e4...|{"visits": [{"url...|
|     F|25-34|d512e295-6a85-491...|{"visits": [{"url...|
|     M|25-34|d51441ea-9dda-454...|{"visits": [{"url...|
|     F|25-34|d51822d4-105b-457...|{"visits": [{"url...|
|     F|35-44|d5183db2-c8e5-413

In [19]:
raw_train.count()

41138

In [20]:
raw_train.select("uid").distinct().count()

41138

In [21]:
raw_train.rdd.getNumPartitions()

5

In [22]:
raw_train = raw_train.repartition(10)

In [23]:
raw_train.rdd.getNumPartitions()

10

In [24]:
raw_train.filter(f.isnan("gender") | f.isnan("age") | f.isnan("uid") | f.isnan("user_json")).count()

0

In [25]:
raw_train.filter(f.isnull("gender") | f.isnull("age") | f.isnull("uid") | f.isnull("user_json")).count()

0

## gender (target_1)

In [26]:
raw_train.show()

+------+-----+--------------------+--------------------+
|gender|  age|                 uid|           user_json|
+------+-----+--------------------+--------------------+
|     M|25-34|3fa85765-ec73-40e...|{"visits": [{"url...|
|     F|35-44|5f1591f6-e044-4df...|{"visits": [{"url...|
|     M|35-44|0a62119d-0f82-4a9...|{"visits": [{"url...|
|     M|25-34|9770847d-c791-425...|{"visits": [{"url...|
|     F| >=55|6765251d-c22c-443...|{"visits": [{"url...|
|     F|25-34|192f39ee-f60b-4e9...|{"visits": [{"url...|
|     M|35-44|407ade9c-8c3e-490...|{"visits": [{"url...|
|     M|25-34|cf56fe24-dd5d-4cd...|{"visits": [{"url...|
|     M|25-34|a1800dd3-159a-4c4...|{"visits": [{"url...|
|     F|35-44|a7f59a9c-f403-420...|{"visits": [{"url...|
|     F|35-44|98162814-295f-48b...|{"visits": [{"url...|
|     F|18-24|a205dcba-ba75-4e4...|{"visits": [{"url...|
|     M|35-44|d070e0ce-93c1-42e...|{"visits": [{"url...|
|     M|35-44|063b0c3e-5ce3-4da...|{"visits": [{"url...|
|     F|35-44|4921e8f9-5e34-4a6

In [27]:
raw_train.groupby("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|17440|
|     M|18698|
|     -| 5000|
+------+-----+



In [28]:
@f.pandas_udf(IntegerType())
def encode_gender(gender):
    mapping = {
        'F': 0,
        'M': 1
    }
    return gender.apply(lambda x: mapping.get(x))

In [29]:
raw_train.withColumn("gender_target", encode_gender("gender")).groupby("gender_target").count().show()

+-------------+-----+
|gender_target|count|
+-------------+-----+
|         null| 5000|
|            1|18698|
|            0|17440|
+-------------+-----+



In [30]:
raw_train = raw_train.withColumn("gender_target", encode_gender("gender"))

In [31]:
raw_train.show()

+------+-----+--------------------+--------------------+-------------+
|gender|  age|                 uid|           user_json|gender_target|
+------+-----+--------------------+--------------------+-------------+
|     M|25-34|3fa85765-ec73-40e...|{"visits": [{"url...|            1|
|     F|35-44|5f1591f6-e044-4df...|{"visits": [{"url...|            0|
|     M|35-44|0a62119d-0f82-4a9...|{"visits": [{"url...|            1|
|     M|25-34|9770847d-c791-425...|{"visits": [{"url...|            1|
|     F| >=55|6765251d-c22c-443...|{"visits": [{"url...|            0|
|     F|25-34|192f39ee-f60b-4e9...|{"visits": [{"url...|            0|
|     M|35-44|407ade9c-8c3e-490...|{"visits": [{"url...|            1|
|     M|25-34|cf56fe24-dd5d-4cd...|{"visits": [{"url...|            1|
|     M|25-34|a1800dd3-159a-4c4...|{"visits": [{"url...|            1|
|     F|35-44|a7f59a9c-f403-420...|{"visits": [{"url...|            0|
|     F|35-44|98162814-295f-48b...|{"visits": [{"url...|            0|
|     

In [32]:
raw_train.filter(raw_train.gender == "-").show()

+------+---+--------------------+--------------------+-------------+
|gender|age|                 uid|           user_json|gender_target|
+------+---+--------------------+--------------------+-------------+
|     -|  -|db81c257-089f-466...|{"visits": [{"url...|         null|
|     -|  -|13db39d6-00c0-486...|{"visits": [{"url...|         null|
|     -|  -|893309fd-b340-4c2...|{"visits": [{"url...|         null|
|     -|  -|5a2c5594-071b-4a2...|{"visits": [{"url...|         null|
|     -|  -|13939bfd-2d09-46f...|{"visits": [{"url...|         null|
|     -|  -|47592189-791e-44f...|{"visits": [{"url...|         null|
|     -|  -|02c56afd-7903-470...|{"visits": [{"url...|         null|
|     -|  -|fd29c2cc-1242-4f8...|{"visits": [{"url...|         null|
|     -|  -|1ef9f346-e510-4cf...|{"visits": [{"url...|         null|
|     -|  -|024be8b9-76c3-43f...|{"visits": [{"url...|         null|
|     -|  -|b12c11f4-542b-4ae...|{"visits": [{"url...|         null|
|     -|  -|02991592-4035-4d6...|{

In [33]:
raw_train = raw_train.filter(~ (raw_train.gender == "-"))

In [34]:
raw_train.groupby("gender_target").count().show()

+-------------+-----+
|gender_target|count|
+-------------+-----+
|            1|18698|
|            0|17440|
+-------------+-----+



In [35]:
raw_train.count()

36138

In [36]:
raw_train.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- user_json: string (nullable = true)
 |-- gender_target: integer (nullable = true)



## age (target_2)

In [37]:
@f.pandas_udf(IntegerType())
def encode_age(age):
    mapping = {
        '18-24': 0,
        '25-34': 1,
        '35-44': 2,
        '45-54': 3,
        '>=55': 4
    }
    return age.apply(lambda x: mapping.get(x))

In [38]:
raw_train = raw_train.withColumn("age_target", encode_age("age"))

In [39]:
raw_train.show()

+------+-----+--------------------+--------------------+-------------+----------+
|gender|  age|                 uid|           user_json|gender_target|age_target|
+------+-----+--------------------+--------------------+-------------+----------+
|     M|25-34|3fa85765-ec73-40e...|{"visits": [{"url...|            1|         1|
|     F|35-44|5f1591f6-e044-4df...|{"visits": [{"url...|            0|         2|
|     M|35-44|0a62119d-0f82-4a9...|{"visits": [{"url...|            1|         2|
|     M|25-34|9770847d-c791-425...|{"visits": [{"url...|            1|         1|
|     F| >=55|6765251d-c22c-443...|{"visits": [{"url...|            0|         4|
|     F|25-34|192f39ee-f60b-4e9...|{"visits": [{"url...|            0|         1|
|     M|35-44|407ade9c-8c3e-490...|{"visits": [{"url...|            1|         2|
|     M|25-34|cf56fe24-dd5d-4cd...|{"visits": [{"url...|            1|         1|
|     M|25-34|a1800dd3-159a-4c4...|{"visits": [{"url...|            1|         1|
|     F|35-44|a7

In [40]:
raw_train.groupby("age").count().show()

+-----+-----+
|  age|count|
+-----+-----+
| >=55| 1679|
|45-54| 4744|
|35-44| 9360|
|25-34|15457|
|18-24| 4898|
+-----+-----+



In [41]:
raw_train.groupby("age_target").count().show()

+----------+-----+
|age_target|count|
+----------+-----+
|         1|15457|
|         3| 4744|
|         4| 1679|
|         2| 9360|
|         0| 4898|
+----------+-----+



In [42]:
1679 + 4744 + 9360 + 15457 + 4898

36138

In [43]:
raw_train.filter(raw_train.age == "-").show()

+------+---+---+---------+-------------+----------+
|gender|age|uid|user_json|gender_target|age_target|
+------+---+---+---------+-------------+----------+
+------+---+---+---------+-------------+----------+



In [44]:
raw_train.show()

+------+-----+--------------------+--------------------+-------------+----------+
|gender|  age|                 uid|           user_json|gender_target|age_target|
+------+-----+--------------------+--------------------+-------------+----------+
|     M|25-34|3fa85765-ec73-40e...|{"visits": [{"url...|            1|         1|
|     F|35-44|5f1591f6-e044-4df...|{"visits": [{"url...|            0|         2|
|     M|35-44|0a62119d-0f82-4a9...|{"visits": [{"url...|            1|         2|
|     M|25-34|9770847d-c791-425...|{"visits": [{"url...|            1|         1|
|     F| >=55|6765251d-c22c-443...|{"visits": [{"url...|            0|         4|
|     F|25-34|192f39ee-f60b-4e9...|{"visits": [{"url...|            0|         1|
|     M|35-44|407ade9c-8c3e-490...|{"visits": [{"url...|            1|         2|
|     M|25-34|cf56fe24-dd5d-4cd...|{"visits": [{"url...|            1|         1|
|     M|25-34|a1800dd3-159a-4c4...|{"visits": [{"url...|            1|         1|
|     F|35-44|a7

In [45]:
raw_train = raw_train.select("uid", "user_json", "gender_target", "age_target")

In [46]:
raw_train.show()

+--------------------+--------------------+-------------+----------+
|                 uid|           user_json|gender_target|age_target|
+--------------------+--------------------+-------------+----------+
|3fa85765-ec73-40e...|{"visits": [{"url...|            1|         1|
|5f1591f6-e044-4df...|{"visits": [{"url...|            0|         2|
|0a62119d-0f82-4a9...|{"visits": [{"url...|            1|         2|
|9770847d-c791-425...|{"visits": [{"url...|            1|         1|
|6765251d-c22c-443...|{"visits": [{"url...|            0|         4|
|192f39ee-f60b-4e9...|{"visits": [{"url...|            0|         1|
|407ade9c-8c3e-490...|{"visits": [{"url...|            1|         2|
|cf56fe24-dd5d-4cd...|{"visits": [{"url...|            1|         1|
|a1800dd3-159a-4c4...|{"visits": [{"url...|            1|         1|
|a7f59a9c-f403-420...|{"visits": [{"url...|            0|         2|
|98162814-295f-48b...|{"visits": [{"url...|            0|         2|
|a205dcba-ba75-4e4...|{"visits": [

In [47]:
raw_train.printSchema()

root
 |-- uid: string (nullable = true)
 |-- user_json: string (nullable = true)
 |-- gender_target: integer (nullable = true)
 |-- age_target: integer (nullable = true)



## user_json (feature)

In [48]:
from pyspark.sql.functions import *

In [49]:
user_json_schema = t.StructType([
    t.StructField('visits', t.ArrayType(
        t.StructType([
            t.StructField('url', t.StringType(), True),
            t.StructField('timestamp', t.LongType(), True)
        ])
    ))
])

In [50]:
raw_train = raw_train.select("uid", "gender_target", "age_target",\
                            from_json(col("user_json"), user_json_schema).alias("s")).\
                            select("uid", col("s.*"), "gender_target", "age_target")

In [51]:
raw_train.show()

+--------------------+--------------------+-------------+----------+
|                 uid|              visits|gender_target|age_target|
+--------------------+--------------------+-------------+----------+
|3fa85765-ec73-40e...|[[http://www.dns-...|            1|         1|
|5f1591f6-e044-4df...|[[http://nrastro....|            0|         2|
|0a62119d-0f82-4a9...|[[http://copypast...|            1|         2|
|9770847d-c791-425...|[[http://filmsonl...|            1|         1|
|6765251d-c22c-443...|[[http://www.msn....|            0|         4|
|192f39ee-f60b-4e9...|[[http://en.lyrse...|            0|         1|
|407ade9c-8c3e-490...|[[http://zaycev.n...|            1|         2|
|cf56fe24-dd5d-4cd...|[[http://razvitie...|            1|         1|
|a1800dd3-159a-4c4...|[[http://vk.com/f...|            1|         1|
|a7f59a9c-f403-420...|[[http://b.jaymed...|            0|         2|
|98162814-295f-48b...|[[http://http://o...|            0|         2|
|a205dcba-ba75-4e4...|[[http://sho

In [52]:
raw_train.printSchema()

root
 |-- uid: string (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: string (nullable = true)
 |    |    |-- timestamp: long (nullable = true)
 |-- gender_target: integer (nullable = true)
 |-- age_target: integer (nullable = true)



In [53]:
raw_train.count()

36138

In [56]:
raw_train_unp = raw_train.select("uid", f.explode("visits").alias("url_list"))

In [57]:
raw_train_unp.count()

5312952

In [58]:
df_url_col = raw_train_unp.select(col("uid"), col("url_list.*"))

In [59]:
df_url_col.show()

+--------------------+--------------------+-------------+
|                 uid|                 url|    timestamp|
+--------------------+--------------------+-------------+
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419923719358|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419923655472|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419923601214|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419923502509|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419923126429|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419888585917|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419882777304|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419879632726|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419879507056|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419877677758|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419877576353|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419877576329|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419877528991|
|3fcfaf7a-8dbc-43d...|http://live.russi...|1419877451196|
|3fcfaf7a-8dbc

In [60]:
df_url_col = df_url_col.select("uid", "url")

In [61]:
df_url_col.show()

+--------------------+--------------------+
|                 uid|                 url|
+--------------------+--------------------+
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://live.russi...|
|3fcfaf7a-8dbc-43d...|http://liv

In [62]:
df_url_col.filter(f.isnull("url")).count()

0

In [63]:
df_url_col.count()

5312952

In [64]:
df_short_url = df_url_col.selectExpr("*", "parse_url(url, 'HOST')").withColumnRenamed('parse_url(url, HOST)', 'short_url')

In [65]:
df_short_url.show()

+--------------------+--------------------+--------------+
|                 uid|                 url|     short_url|
+--------------------+--------------------+--------------+
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.t

In [66]:
df_short_url.filter(f.isnull("url")).count()

0

In [67]:
df_short_url.filter(f.isnull("short_url")).count()

128051

In [68]:
( 128051 / 5312952 ) * 100

2.4101667020518915

In [69]:
df_short_url.filter(f.isnull("short_url")).show(1000)

+--------------------+--------------------+---------+
|                 uid|                 url|short_url|
+--------------------+--------------------+---------+
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http://www.nudevi...|     null|
|3fcfaf7a-8dbc-43d...|http:/

In [70]:
df_short_url = df_short_url.filter(~f.isnull("short_url"))

In [71]:
df_short_url.show()

+--------------------+--------------------+--------------+
|                 uid|                 url|     short_url|
+--------------------+--------------------+--------------+
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.tv|
|3fcfaf7a-8dbc-43d...|http://live.russi...|live.russia.t

In [72]:
df_short_url.count()

5184901

In [73]:
df_short_url = df_short_url.groupBy("uid").agg(collect_list("short_url").alias("short_url_list"))

In [74]:
df_short_url.show()

+--------------------+--------------------+
|                 uid|      short_url_list|
+--------------------+--------------------+
|0392f398-ea7e-4a1...|[www.adme.ru, www...|
|094b1e7e-97a6-441...|[kinoclips.net, n...|
|095544a2-64f7-422...|[well-good.ru, we...|
|098a0e00-8597-475...|[www.youtube.com,...|
|0a595fa1-bae0-41d...|[go.mail.ru, go.m...|
|0a6868f4-2cda-425...|[mirtankov.su, wo...|
|0b36fd6b-00bf-483...|[prominator.ru, p...|
|0b4db476-388d-43b...|[rodovid.me, www....|
|0bb5e302-6ec5-49e...|[bigbuzzy.ru, big...|
|0f5f92bd-a956-487...|[mail.rambler.ru,...|
|1155961f-7ce1-453...|[www.aif.ru, yand...|
|1517b5ce-f67f-499...|        [kinogo.net]|
|1729bdad-6b6c-474...|[portirkutsk.ru, ...|
|17a16d9d-915d-46a...|[l.facebook.com, ...|
|1a36e2c1-f263-455...|[www.brazzers.com...|
|1a6423a8-ffec-49d...|[www.dissercat.co...|
|1ba0d2cb-e4b8-4a1...|[admiralgameslots...|
|1bc25004-16a1-454...|[bigpicture.ru, b...|
|1f0269d0-810a-42a...|[ru.euronews.com,...|
|1fdedd82-ba5c-4e0...|[kino-file

In [75]:
df_short_url.count()

35960

In [76]:
36138 - 35960

178

In [77]:
(178 / 36138 ) * 100

0.4925563119154353

In [78]:
df_short_url.filter(f.isnull("short_url_list")).count()

0

In [79]:
raw_train.show()

+--------------------+--------------------+-------------+----------+
|                 uid|              visits|gender_target|age_target|
+--------------------+--------------------+-------------+----------+
|3fa85765-ec73-40e...|[[http://www.dns-...|            1|         1|
|5f1591f6-e044-4df...|[[http://nrastro....|            0|         2|
|0a62119d-0f82-4a9...|[[http://copypast...|            1|         2|
|9770847d-c791-425...|[[http://filmsonl...|            1|         1|
|6765251d-c22c-443...|[[http://www.msn....|            0|         4|
|192f39ee-f60b-4e9...|[[http://en.lyrse...|            0|         1|
|407ade9c-8c3e-490...|[[http://zaycev.n...|            1|         2|
|cf56fe24-dd5d-4cd...|[[http://razvitie...|            1|         1|
|a1800dd3-159a-4c4...|[[http://vk.com/f...|            1|         1|
|a7f59a9c-f403-420...|[[http://b.jaymed...|            0|         2|
|98162814-295f-48b...|[[http://http://o...|            0|         2|
|a205dcba-ba75-4e4...|[[http://sho

In [80]:
raw_train.count()

36138

In [81]:
train_short_url = raw_train.join(df_short_url, on="uid", how="inner")

In [82]:
train_short_url.show()

+--------------------+--------------------+-------------+----------+--------------------+
|                 uid|              visits|gender_target|age_target|      short_url_list|
+--------------------+--------------------+-------------+----------+--------------------+
|0392f398-ea7e-4a1...|[[http://www.adme...|            0|         4|[www.adme.ru, www...|
|094b1e7e-97a6-441...|[[http://kinoclip...|            0|         0|[kinoclips.net, n...|
|095544a2-64f7-422...|[[http://well-goo...|            0|         1|[well-good.ru, we...|
|098a0e00-8597-475...|[[http://www.yout...|            0|         4|[www.youtube.com,...|
|0a595fa1-bae0-41d...|[[http://go.mail....|            1|         4|[go.mail.ru, go.m...|
|0a6868f4-2cda-425...|[[http://mirtanko...|            0|         0|[mirtankov.su, wo...|
|0b36fd6b-00bf-483...|[[http://prominat...|            0|         0|[prominator.ru, p...|
|0b4db476-388d-43b...|[[http://rodovid....|            1|         2|[rodovid.me, www....|
|0bb5e302-

In [85]:
train_short_url.filter(f.isnull("short_url_list")).count()

0

In [83]:
train_short_url.count()

35960

In [86]:
train_short_url.printSchema()

root
 |-- uid: string (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: string (nullable = true)
 |    |    |-- timestamp: long (nullable = true)
 |-- gender_target: integer (nullable = true)
 |-- age_target: integer (nullable = true)
 |-- short_url_list: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [87]:
hasher = HashingTF(inputCol="short_url_list", outputCol="features", numFeatures=10000)
train_df = hasher.transform(train_short_url)

In [88]:
train_df.show()

+--------------------+--------------------+-------------+----------+--------------------+--------------------+
|                 uid|              visits|gender_target|age_target|      short_url_list|            features|
+--------------------+--------------------+-------------+----------+--------------------+--------------------+
|0392f398-ea7e-4a1...|[[http://www.adme...|            0|         4|[www.adme.ru, www...|(10000,[181,267,3...|
|094b1e7e-97a6-441...|[[http://kinoclip...|            0|         0|[kinoclips.net, n...|(10000,[1362,1501...|
|095544a2-64f7-422...|[[http://well-goo...|            0|         1|[well-good.ru, we...|(10000,[8149],[4.0])|
|098a0e00-8597-475...|[[http://www.yout...|            0|         4|[www.youtube.com,...|(10000,[5458,6021...|
|0a595fa1-bae0-41d...|[[http://go.mail....|            1|         4|[go.mail.ru, go.m...|(10000,[2094,4925...|
|0a6868f4-2cda-425...|[[http://mirtanko...|            0|         0|[mirtankov.su, wo...|(10000,[993,1790,...|
|

In [89]:
train_df = train_df.select("uid", "gender_target", "age_target", "features")

In [90]:
train_df.show()

+--------------------+-------------+----------+--------------------+
|                 uid|gender_target|age_target|            features|
+--------------------+-------------+----------+--------------------+
|0392f398-ea7e-4a1...|            0|         4|(10000,[181,267,3...|
|094b1e7e-97a6-441...|            0|         0|(10000,[1362,1501...|
|095544a2-64f7-422...|            0|         1|(10000,[8149],[4.0])|
|098a0e00-8597-475...|            0|         4|(10000,[5458,6021...|
|0a595fa1-bae0-41d...|            1|         4|(10000,[2094,4925...|
|0a6868f4-2cda-425...|            0|         0|(10000,[993,1790,...|
|0b36fd6b-00bf-483...|            0|         0|(10000,[498,891,4...|
|0b4db476-388d-43b...|            1|         2|(10000,[1531,1659...|
|0bb5e302-6ec5-49e...|            1|         0|(10000,[151,2071,...|
|0f5f92bd-a956-487...|            0|         3|(10000,[225,1475,...|
|1155961f-7ce1-453...|            0|         2|(10000,[6151,6582...|
|1517b5ce-f67f-499...|            

In [91]:
train_df.printSchema()

root
 |-- uid: string (nullable = true)
 |-- gender_target: integer (nullable = true)
 |-- age_target: integer (nullable = true)
 |-- features: vector (nullable = true)



In [92]:
train_df.count()

35960

In [95]:
train_df.groupby("gender_target").count().show()

+-------------+-----+
|gender_target|count|
+-------------+-----+
|            1|18623|
|            0|17337|
+-------------+-----+



In [96]:
train_df.groupby("age_target").count().show()

+----------+-----+
|age_target|count|
+----------+-----+
|         1|15378|
|         3| 4730|
|         4| 1671|
|         2| 9325|
|         0| 4856|
+----------+-----+



In [98]:
train_df.select("features").show(1000, 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                            features|
+----------------------------------------------------------------------------------------------------+
|(10000,[181,267,319,616,665,1087,1493,1497,1526,1789,1932,2094,2231,2277,2418,2766,3204,3275,3321...|
|(10000,[1362,1501,1631,1987,2250,3204,3913,4485,6657,6969,7392,8180,9604,9770,9922],[1.0,1.0,1.0,...|
|                                                                                (10000,[8149],[4.0])|
|                                                                       (10000,[5458,6021],[1.0,1.0])|
|                                                    (10000,[2094,4925,4988,9037],[8.0,1.0,11.0,1.0])|
|                                             (10000,[993,1790,3672,4988,5181],[2.0,8.0,1.0,1.0,1.0])|
|                                                                (10000,[

# MODEL GENDER

## train_gender

In [99]:
train_gender = train_df.select("uid", "gender_target", "features")

In [100]:
train_gender.groupby("gender_target").count().collect()

[Row(gender_target=1, count=18623), Row(gender_target=0, count=17337)]

In [101]:
train_gender.show()

+--------------------+-------------+--------------------+
|                 uid|gender_target|            features|
+--------------------+-------------+--------------------+
|0392f398-ea7e-4a1...|            0|(10000,[181,267,3...|
|094b1e7e-97a6-441...|            0|(10000,[1362,1501...|
|095544a2-64f7-422...|            0|(10000,[8149],[4.0])|
|098a0e00-8597-475...|            0|(10000,[5458,6021...|
|0a595fa1-bae0-41d...|            1|(10000,[2094,4925...|
|0a6868f4-2cda-425...|            0|(10000,[993,1790,...|
|0b36fd6b-00bf-483...|            0|(10000,[498,891,4...|
|0b4db476-388d-43b...|            1|(10000,[1531,1659...|
|0bb5e302-6ec5-49e...|            1|(10000,[151,2071,...|
|0f5f92bd-a956-487...|            0|(10000,[225,1475,...|
|1155961f-7ce1-453...|            0|(10000,[6151,6582...|
|1517b5ce-f67f-499...|            0|(10000,[6205],[1.0])|
|1729bdad-6b6c-474...|            0|(10000,[665,979,1...|
|17a16d9d-915d-46a...|            1|(10000,[439,2076,...|
|1a36e2c1-f263

In [102]:
train_gender.count()

35960

## rfc_gender_model

In [103]:
rfc_gender = RandomForestClassifier(featuresCol="features", labelCol="gender_target", maxDepth=5, numTrees=200)

In [105]:
rfc_gender_model = rfc_gender.fit(train_gender)

In [106]:
rfc_gender_model

RandomForestClassificationModel (uid=RandomForestClassifier_feee6dfc0008) with 200 trees

# MODEL AGE

## train_age

In [107]:
train_age = train_df.select("uid", "age_target", "features")

In [108]:
train_age.groupby("age_target").count().collect()

[Row(age_target=1, count=15378),
 Row(age_target=3, count=4730),
 Row(age_target=4, count=1671),
 Row(age_target=2, count=9325),
 Row(age_target=0, count=4856)]

In [109]:
train_age.show()

+--------------------+----------+--------------------+
|                 uid|age_target|            features|
+--------------------+----------+--------------------+
|0392f398-ea7e-4a1...|         4|(10000,[181,267,3...|
|094b1e7e-97a6-441...|         0|(10000,[1362,1501...|
|095544a2-64f7-422...|         1|(10000,[8149],[4.0])|
|098a0e00-8597-475...|         4|(10000,[5458,6021...|
|0a595fa1-bae0-41d...|         4|(10000,[2094,4925...|
|0a6868f4-2cda-425...|         0|(10000,[993,1790,...|
|0b36fd6b-00bf-483...|         0|(10000,[498,891,4...|
|0b4db476-388d-43b...|         2|(10000,[1531,1659...|
|0bb5e302-6ec5-49e...|         0|(10000,[151,2071,...|
|0f5f92bd-a956-487...|         3|(10000,[225,1475,...|
|1155961f-7ce1-453...|         2|(10000,[6151,6582...|
|1517b5ce-f67f-499...|         1|(10000,[6205],[1.0])|
|1729bdad-6b6c-474...|         2|(10000,[665,979,1...|
|17a16d9d-915d-46a...|         2|(10000,[439,2076,...|
|1a36e2c1-f263-455...|         1|(10000,[1943,6878...|
|1a6423a8-

In [110]:
train_age.count()

35960

## rfc_age_model

In [111]:
rfc_age = RandomForestClassifier(featuresCol="features", labelCol="age_target", maxDepth=5, numTrees=200)

In [112]:
rfc_age_model = rfc_age.fit(train_age)

In [113]:
rfc_age_model

RandomForestClassificationModel (uid=RandomForestClassifier_d83973c7afd7) with 200 trees

# Kafka

In [114]:
KAFKA_BOOTSTRAP_SERVER = 'spark-node-1.newprolab.com:6667'
KAFKA_INPUT_TOPIC = 'input_darina.bocharova'
KAFKA_OUTPUT_TOPIC = 'darina.bocharova'

In [115]:
kafka_read_df = (
    spark.read
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('subscribe', KAFKA_INPUT_TOPIC)
    .option('startingOffsets', 'earliest')
    .option('failOnDataLoss', 'False')
    .load()
    .cache()
)

In [116]:
kafka_read_df

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [119]:
kafka_read_df.show(6000)

+----+--------------------+--------------------+---------+------+--------------------+-------------+
| key|               value|               topic|partition|offset|           timestamp|timestampType|
+----+--------------------+--------------------+---------+------+--------------------+-------------+
|null|[7B 22 75 69 64 2...|input_darina.boch...|        0|     0|2022-11-01 12:46:...|            0|
|null|[7B 22 75 69 64 2...|input_darina.boch...|        0|     1|2022-11-01 12:46:...|            0|
|null|[7B 22 75 69 64 2...|input_darina.boch...|        0|     2|2022-11-01 12:46:...|            0|
|null|[7B 22 75 69 64 2...|input_darina.boch...|        0|     3|2022-11-01 12:46:...|            0|
|null|[7B 22 75 69 64 2...|input_darina.boch...|        0|     4|2022-11-01 12:46:...|            0|
|null|[7B 22 75 69 64 2...|input_darina.boch...|        0|     5|2022-11-01 12:46:...|            0|
|null|[7B 22 75 69 64 2...|input_darina.boch...|        0|     6|2022-11-01 12:46:...|     

In [228]:
kafka_read_df = (
    spark.read
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('subscribe', KAFKA_INPUT_TOPIC)
    .option('startingOffsets', '{ "input_darina.bocharova": {"0": 20000}}')
    .option('failOnDataLoss', 'False')
    .load()
    .cache()
)

In [229]:
kafka_read_df.count()

5000

In [231]:
event_schema = t.StructType([
    t.StructField('uid', t.StringType(), True),
    t.StructField('visits', t.StringType(), True),
])

In [232]:
visit_schema = t.ArrayType(
    t.StructType([
        t.StructField('url', t.StringType(), True),
        t.StructField('timestamp', t.LongType(), True)
    ])
)

In [233]:
clean_df = (
    kafka_read_df
    .select(f.col('value').cast('string').alias('value'))
    .select(f.from_json(f.col('value'), event_schema).alias('event'))
    .select(
        'event.uid', 
        f.from_json(f.col('event.visits'), visit_schema).alias('visits')
    )
)

In [234]:
clean_df.show(20, 70)

+------------------------------------+----------------------------------------------------------------------+
|                                 uid|                                                                visits|
+------------------------------------+----------------------------------------------------------------------+
|bd7a30e1-a25d-4cbf-a03f-61748cbe540e|[[http://www.interfax.ru/business/414668, 1419775945781], [http://a...|
|bd7a6f52-45db-49bf-90f2-a3b07a9b7bcd|[[https://www.packagetrackr.com/track/ups, 1419920763569], [https:/...|
|bd7a7fd9-ab06-42f5-bf0f-1cbb0463004c|[[http://www.mk.ru/incident/2015/02/27/latviya-podnyala-v-vozdukh-i...|
|bd7c5d7a-0def-41d1-895f-fdb96c56c2d4|[[http://www.24open.ru/user/elena8020445/, 1426820740000], [http://...|
|bd7e54a2-0215-45cb-a869-9efebf250e38|[[http://www.dns-shop.ru/catalog/i172806/tverdotelnyj-nakopitel-ssd...|
|bd7e9797-4cdb-46e1-a540-f3ea010605ad|[[http://news.meta.ua/cluster:41878362-v-sevastopole-nashli-dokhlui...|
|bd7e9ec7-

In [235]:
clean_df.count()

5000

In [237]:
clean_df.printSchema()

root
 |-- uid: string (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: string (nullable = true)
 |    |    |-- timestamp: long (nullable = true)



In [238]:
kafka_url = clean_df.select("uid", f.explode("visits").alias("url_list"))

In [239]:
kafka_url.count()

516555

In [240]:
kafka_url.printSchema()

root
 |-- uid: string (nullable = true)
 |-- url_list: struct (nullable = true)
 |    |-- url: string (nullable = true)
 |    |-- timestamp: long (nullable = true)



In [241]:
kafka_url = kafka_url.select(col("uid"), col("url_list.*"))

In [242]:
kafka_url.show()

+--------------------+--------------------+-------------+
|                 uid|                 url|    timestamp|
+--------------------+--------------------+-------------+
|bd7a30e1-a25d-4cb...|http://www.interf...|1419775945781|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419679865088|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419679853405|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419679847246|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419679822992|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419679749670|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419679738132|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419679728982|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419679720228|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419679713288|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419679687135|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419676057723|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419676052200|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|1419676031461|
|bd7a30e1-a25d

In [243]:
kafka_url = kafka_url.select("uid", "url")

In [244]:
kafka_url.show()

+--------------------+--------------------+
|                 uid|                 url|
+--------------------+--------------------+
|bd7a30e1-a25d-4cb...|http://www.interf...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|
|bd7a30e1-a25d-4cb...|http://tv.jampo.t...|
|bd7a30e1-a25d-4cb...|http://www.sovsek...|
|bd7a30e1-a25d-4cb...|http://mar

In [245]:
kafka_sort_url = kafka_url.selectExpr("*", "parse_url(url, 'HOST')").withColumnRenamed('parse_url(url, HOST)', 'short_url')

In [246]:
kafka_sort_url.show()

+--------------------+--------------------+--------------------+
|                 uid|                 url|           short_url|
+--------------------+--------------------+--------------------+
|bd7a30e1-a25d-4cb...|http://www.interf...|     www.interfax.ru|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|http://amerikan-g...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|htt

In [247]:
kafka_sort_url.count()

516555

In [248]:
kafka_sort_url.filter(f.isnull("short_url")).count()

13792

In [249]:
f_kafka_sort_url = kafka_sort_url.filter(~f.isnull("short_url"))

In [250]:
f_kafka_sort_url = f_kafka_sort_url.select("uid", "short_url")

In [251]:
f_kafka_sort_url.count()

502763

In [252]:
f_kafka_sort_url.show()

+--------------------+--------------------+
|                 uid|           short_url|
+--------------------+--------------------+
|bd7a30e1-a25d-4cb...|     www.interfax.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|amerikan-gruzovik.ru|
|bd7a30e1-a25d-4cb...|         tv.jampo.tv|
|bd7a30e1-a25d-4cb...|  www.sovsekretno.ru|
|bd7a30e1-a25d-4cb...|       mar

In [253]:
kafka_sort_url_agg = f_kafka_sort_url.groupBy("uid").agg(collect_list("short_url").alias("short_url_list"))

In [254]:
kafka_sort_url_agg.show(20, 70)

+------------------------------------+----------------------------------------------------------------------+
|                                 uid|                                                        short_url_list|
+------------------------------------+----------------------------------------------------------------------+
|0108d217-e476-493d-8c81-a9744f12451a|                            [kvartblog.ru, kvartblog.ru, kvartblog.ru]|
|0192cc54-559c-4c8e-89b4-5f4bf31e4245|[metanol.lv, metanol.lv, metanol.lv, metanol.lv, metanol.lv, metano...|
|019acd5e-be9a-4cde-8280-0d2376dc016d|[www.russianfood.com, www.russianfood.com, www.russianfood.com, www...|
|02e7f830-da57-4d57-a916-10ff45f5de7b|[maxpark.com, newsland.com, newsland.com, www.google.ru, www.conspi...|
|1d160259-73d8-4514-911e-757042b22812|                                                      [ua.sinoptik.ua]|
|1e14a504-276e-4489-b5c5-481d3ea9097f|[www.rusfishing.ru, www.gismeteo.ru, www.rusfishing.ru, news.sportb...|
|1eb313db-

In [255]:
kafka_sort_url_agg.count()

4976

In [256]:
kafka_sort_url_agg.filter(f.isnull("short_url_list")).count()

0

In [257]:
val_df = clean_df.join(kafka_sort_url_agg, on="uid", how="inner")

In [258]:
val_df.show()

+--------------------+--------------------+--------------------+
|                 uid|              visits|      short_url_list|
+--------------------+--------------------+--------------------+
|bd7a30e1-a25d-4cb...|[[http://www.inte...|[www.interfax.ru,...|
|bd7a6f52-45db-49b...|[[https://www.pac...|[www.packagetrack...|
|bd7a7fd9-ab06-42f...|[[http://www.mk.r...|[www.mk.ru, www.m...|
|bd7c5d7a-0def-41d...|[[http://www.24op...|[www.24open.ru, w...|
|bd7e54a2-0215-45c...|[[http://www.dns-...|[www.dns-shop.ru,...|
|bd7e9797-4cdb-46e...|[[http://news.met...|      [news.meta.ua]|
|bd7e9ec7-fb67-45e...|[[http://dynamobr...|[dynamobryansk.fo...|
|bd8056df-cc25-4b6...|[[http://www.2mm....|[www.2mm.ru, www....|
|bd818690-73d2-445...|[[http://www.lacy...|[www.lacywear.ru,...|
|bd81e006-f059-4cd...|[[http://nn.domru...|       [nn.domru.ru]|
|bd81e64a-bfa3-414...|[[http://cache.be...|[cache.betweendig...|
|bd82fee4-afb3-408...|[[http://apostrop...|[apostrophe.com.u...|
|bd83400b-abe2-42f...|[[h

In [259]:
val_df.count()

4976

In [260]:
val_df.filter(f.isnull("short_url_list")).count()

0

In [261]:
val_df = hasher.transform(val_df)

In [262]:
val_df.show()

+--------------------+--------------------+--------------------+--------------------+
|                 uid|              visits|      short_url_list|            features|
+--------------------+--------------------+--------------------+--------------------+
|bd7a30e1-a25d-4cb...|[[http://www.inte...|[www.interfax.ru,...|(10000,[162,210,2...|
|bd7a6f52-45db-49b...|[[https://www.pac...|[www.packagetrack...|(10000,[6,58,64,8...|
|bd7a7fd9-ab06-42f...|[[http://www.mk.r...|[www.mk.ru, www.m...|(10000,[1482,6925...|
|bd7c5d7a-0def-41d...|[[http://www.24op...|[www.24open.ru, w...|(10000,[304,658,1...|
|bd7e54a2-0215-45c...|[[http://www.dns-...|[www.dns-shop.ru,...|(10000,[560,979,1...|
|bd7e9797-4cdb-46e...|[[http://news.met...|      [news.meta.ua]|(10000,[9055],[1.0])|
|bd7e9ec7-fb67-45e...|[[http://dynamobr...|[dynamobryansk.fo...|(10000,[6587],[1.0])|
|bd8056df-cc25-4b6...|[[http://www.2mm....|[www.2mm.ru, www....|(10000,[3424,5465...|
|bd818690-73d2-445...|[[http://www.lacy...|[www.lacywe

In [263]:
val_df = val_df.select("uid", "features")

In [264]:
val_df.show()

+--------------------+--------------------+
|                 uid|            features|
+--------------------+--------------------+
|0108d217-e476-493...|(10000,[1257],[3.0])|
|0192cc54-559c-4c8...|(10000,[120,1058,...|
|019acd5e-be9a-4cd...|(10000,[2155,3273...|
|02e7f830-da57-4d5...|(10000,[1186,5874...|
|1d160259-73d8-451...|(10000,[2783],[1.0])|
|1e14a504-276e-448...|(10000,[2304,3016...|
|1eb313db-34ff-4bf...|(10000,[7792],[7.0])|
|1eff6e4f-3b8a-447...|(10000,[1550],[2.0])|
|3e75c432-cb78-488...|(10000,[1904,3472...|
|47565df3-13e3-460...|(10000,[266,3166,...|
|4766a8ab-e9b6-4e0...|(10000,[979,1856,...|
|50637c81-fffa-4ee...|(10000,[1562,4965...|
|5a023519-f28e-4eb...|(10000,[1001,4910...|
|5a781caa-6131-4d9...| (10000,[907],[1.0])|
|5ab3c7b8-c550-493...|(10000,[261,979,1...|
|7302e78a-ec04-47e...|(10000,[40,69,104...|
|73081df3-8f41-435...|(10000,[2495,3757...|
|89fe85cb-ea4c-4be...|(10000,[4412,4886...|
|8affa6ce-24c7-4ed...|(10000,[771,5195,...|
|b2e4450d-c582-441...|(10000,[10

In [265]:
val_df.count()

4976

## kafka_predict_gender

In [267]:
kafka_predict_gender = rfc_gender_model.transform(val_df)

In [268]:
kafka_predict_gender.select("uid", "features", "probability", "prediction").show()

+--------------------+--------------------+--------------------+----------+
|                 uid|            features|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|0108d217-e476-493...|(10000,[1257],[3.0])|[0.48866261719110...|       1.0|
|0192cc54-559c-4c8...|(10000,[120,1058,...|[0.48866261719110...|       1.0|
|019acd5e-be9a-4cd...|(10000,[2155,3273...|[0.50237116030479...|       0.0|
|02e7f830-da57-4d5...|(10000,[1186,5874...|[0.48866261719110...|       1.0|
|1d160259-73d8-451...|(10000,[2783],[1.0])|[0.48866261719110...|       1.0|
|1e14a504-276e-448...|(10000,[2304,3016...|[0.48014319748998...|       1.0|
|1eb313db-34ff-4bf...|(10000,[7792],[7.0])|[0.48866261719110...|       1.0|
|1eff6e4f-3b8a-447...|(10000,[1550],[2.0])|[0.48866261719110...|       1.0|
|3e75c432-cb78-488...|(10000,[1904,3472...|[0.49749262863984...|       1.0|
|47565df3-13e3-460...|(10000,[266,3166,...|[0.48866261719110...|       1.0|
|4766a8ab-e9

In [269]:
kafka_predict_gender.filter(f.isnull("prediction")).count()

0

In [270]:
kafka_predict_gender = kafka_predict_gender.select("uid", "prediction").withColumnRenamed('prediction', 'prediction_gender')

In [271]:
kafka_predict_gender.show()

+--------------------+-----------------+
|                 uid|prediction_gender|
+--------------------+-----------------+
|0108d217-e476-493...|              1.0|
|0192cc54-559c-4c8...|              1.0|
|019acd5e-be9a-4cd...|              0.0|
|02e7f830-da57-4d5...|              1.0|
|1d160259-73d8-451...|              1.0|
|1e14a504-276e-448...|              1.0|
|1eb313db-34ff-4bf...|              1.0|
|1eff6e4f-3b8a-447...|              1.0|
|3e75c432-cb78-488...|              1.0|
|47565df3-13e3-460...|              1.0|
|4766a8ab-e9b6-4e0...|              1.0|
|50637c81-fffa-4ee...|              1.0|
|5a023519-f28e-4eb...|              1.0|
|5a781caa-6131-4d9...|              1.0|
|5ab3c7b8-c550-493...|              1.0|
|7302e78a-ec04-47e...|              1.0|
|73081df3-8f41-435...|              1.0|
|89fe85cb-ea4c-4be...|              1.0|
|8affa6ce-24c7-4ed...|              1.0|
|b2e4450d-c582-441...|              1.0|
+--------------------+-----------------+
only showing top

In [272]:
kafka_predict_gender.count()

4976

In [273]:
kafka_predict_gender.printSchema()

root
 |-- uid: string (nullable = true)
 |-- prediction_gender: double (nullable = false)



## kafka_predict_age

In [275]:
kafka_predict_age = rfc_age_model.transform(val_df)

In [276]:
kafka_predict_age.select("uid", "features", "probability", "prediction").show()

+--------------------+--------------------+--------------------+----------+
|                 uid|            features|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|0108d217-e476-493...|(10000,[1257],[3.0])|[0.13896561039656...|       1.0|
|0192cc54-559c-4c8...|(10000,[120,1058,...|[0.13896561039656...|       1.0|
|019acd5e-be9a-4cd...|(10000,[2155,3273...|[0.13909823024917...|       1.0|
|02e7f830-da57-4d5...|(10000,[1186,5874...|[0.13922547283537...|       1.0|
|1d160259-73d8-451...|(10000,[2783],[1.0])|[0.13896561039656...|       1.0|
|1e14a504-276e-448...|(10000,[2304,3016...|[0.13896561039656...|       1.0|
|1eb313db-34ff-4bf...|(10000,[7792],[7.0])|[0.13896561039656...|       1.0|
|1eff6e4f-3b8a-447...|(10000,[1550],[2.0])|[0.13178866044982...|       1.0|
|3e75c432-cb78-488...|(10000,[1904,3472...|[0.13437470631629...|       1.0|
|47565df3-13e3-460...|(10000,[266,3166,...|[0.13896561039656...|       1.0|
|4766a8ab-e9

In [277]:
kafka_predict_age.filter(f.isnull("prediction")).count()

0

In [278]:
kafka_predict_age = kafka_predict_age.select("uid", "prediction").withColumnRenamed('prediction', 'prediction_age')

In [279]:
kafka_predict_age.show()

+--------------------+--------------+
|                 uid|prediction_age|
+--------------------+--------------+
|0108d217-e476-493...|           1.0|
|0192cc54-559c-4c8...|           1.0|
|019acd5e-be9a-4cd...|           1.0|
|02e7f830-da57-4d5...|           1.0|
|1d160259-73d8-451...|           1.0|
|1e14a504-276e-448...|           1.0|
|1eb313db-34ff-4bf...|           1.0|
|1eff6e4f-3b8a-447...|           1.0|
|3e75c432-cb78-488...|           1.0|
|47565df3-13e3-460...|           1.0|
|4766a8ab-e9b6-4e0...|           1.0|
|50637c81-fffa-4ee...|           1.0|
|5a023519-f28e-4eb...|           1.0|
|5a781caa-6131-4d9...|           1.0|
|5ab3c7b8-c550-493...|           1.0|
|7302e78a-ec04-47e...|           1.0|
|73081df3-8f41-435...|           1.0|
|89fe85cb-ea4c-4be...|           1.0|
|8affa6ce-24c7-4ed...|           1.0|
|b2e4450d-c582-441...|           1.0|
+--------------------+--------------+
only showing top 20 rows



In [280]:
kafka_predict_age.count()

4976

In [281]:
kafka_predict_age.printSchema()

root
 |-- uid: string (nullable = true)
 |-- prediction_age: double (nullable = false)



## kafka_answer

In [283]:
kafka_answer = kafka_predict_gender.join(kafka_predict_age, on="uid", how="inner")

In [284]:
kafka_answer.show()

+--------------------+-----------------+--------------+
|                 uid|prediction_gender|prediction_age|
+--------------------+-----------------+--------------+
|0108d217-e476-493...|              1.0|           1.0|
|0192cc54-559c-4c8...|              1.0|           1.0|
|019acd5e-be9a-4cd...|              0.0|           1.0|
|02e7f830-da57-4d5...|              1.0|           1.0|
|1d160259-73d8-451...|              1.0|           1.0|
|1e14a504-276e-448...|              1.0|           1.0|
|1eb313db-34ff-4bf...|              1.0|           1.0|
|1eff6e4f-3b8a-447...|              1.0|           1.0|
|3e75c432-cb78-488...|              1.0|           1.0|
|47565df3-13e3-460...|              1.0|           1.0|
|4766a8ab-e9b6-4e0...|              1.0|           1.0|
|50637c81-fffa-4ee...|              1.0|           1.0|
|5a023519-f28e-4eb...|              1.0|           1.0|
|5a781caa-6131-4d9...|              1.0|           1.0|
|5ab3c7b8-c550-493...|              1.0|        

In [285]:
kafka_answer.count()

4976

In [286]:
clean_df.show()

+--------------------+--------------------+
|                 uid|              visits|
+--------------------+--------------------+
|bd7a30e1-a25d-4cb...|[[http://www.inte...|
|bd7a6f52-45db-49b...|[[https://www.pac...|
|bd7a7fd9-ab06-42f...|[[http://www.mk.r...|
|bd7c5d7a-0def-41d...|[[http://www.24op...|
|bd7e54a2-0215-45c...|[[http://www.dns-...|
|bd7e9797-4cdb-46e...|[[http://news.met...|
|bd7e9ec7-fb67-45e...|[[http://dynamobr...|
|bd8056df-cc25-4b6...|[[http://www.2mm....|
|bd818690-73d2-445...|[[http://www.lacy...|
|bd81e006-f059-4cd...|[[http://nn.domru...|
|bd81e64a-bfa3-414...|[[http://cache.be...|
|bd82fee4-afb3-408...|[[http://apostrop...|
|bd83400b-abe2-42f...|[[http://index.ru...|
|bd843c8c-dbba-4ec...|[[http://katushka...|
|bd86d250-a6ee-41f...|[[http://www.sq.c...|
|bd889738-93b2-402...|[[http://www.prof...|
|bd88fac5-3211-439...|[[http://yourlust...|
|bd89a20f-a7af-462...|[[http://yandex.r...|
|bd8c2ee4-9c45-46d...|[[http://video-do...|
|bd8f08a1-d845-4cb...|[[http://l

In [287]:
answer = clean_df.join(kafka_answer, on="uid", how="outer")

In [288]:
answer = answer.select("uid", "prediction_gender", "prediction_age")

In [289]:
answer.show()

+--------------------+-----------------+--------------+
|                 uid|prediction_gender|prediction_age|
+--------------------+-----------------+--------------+
|0108d217-e476-493...|              1.0|           1.0|
|0192cc54-559c-4c8...|              1.0|           1.0|
|019acd5e-be9a-4cd...|              0.0|           1.0|
|02e7f830-da57-4d5...|              1.0|           1.0|
|1d160259-73d8-451...|              1.0|           1.0|
|1e14a504-276e-448...|              1.0|           1.0|
|1eb313db-34ff-4bf...|              1.0|           1.0|
|1eff6e4f-3b8a-447...|              1.0|           1.0|
|3e75c432-cb78-488...|              1.0|           1.0|
|47565df3-13e3-460...|              1.0|           1.0|
|4766a8ab-e9b6-4e0...|              1.0|           1.0|
|50637c81-fffa-4ee...|              1.0|           1.0|
|5a023519-f28e-4eb...|              1.0|           1.0|
|5a781caa-6131-4d9...|              1.0|           1.0|
|5ab3c7b8-c550-493...|              1.0|        

In [290]:
answer.count()

5000

In [292]:
answer = answer.na.fill(value=0,subset=["prediction_gender"])

In [295]:
answer = answer.na.fill(value=0,subset=["prediction_age"])

In [297]:
@f.pandas_udf(StringType())
def decode_age(age):
    mapping = {
        0 : '18-24',
        1 : '25-34',
        2 : '35-44',
        3 : '45-54',
        4 : '>=55'
    }
    return age.apply(lambda x: mapping.get(x))

In [298]:
@f.pandas_udf(StringType())
def decode_gender(gender):
    mapping = {
        0 : 'F',
        1 : 'M'
    }
    return gender.apply(lambda x: mapping.get(x))

In [299]:
answer1 = answer.withColumn("age", decode_age("prediction_age"))

In [300]:
answer1.show()

+--------------------+-----------------+--------------+-----+
|                 uid|prediction_gender|prediction_age|  age|
+--------------------+-----------------+--------------+-----+
|0108d217-e476-493...|              1.0|           1.0|25-34|
|0192cc54-559c-4c8...|              1.0|           1.0|25-34|
|019acd5e-be9a-4cd...|              0.0|           1.0|25-34|
|02e7f830-da57-4d5...|              1.0|           1.0|25-34|
|1d160259-73d8-451...|              1.0|           1.0|25-34|
|1e14a504-276e-448...|              1.0|           1.0|25-34|
|1eb313db-34ff-4bf...|              1.0|           1.0|25-34|
|1eff6e4f-3b8a-447...|              1.0|           1.0|25-34|
|3e75c432-cb78-488...|              1.0|           1.0|25-34|
|47565df3-13e3-460...|              1.0|           1.0|25-34|
|4766a8ab-e9b6-4e0...|              1.0|           1.0|25-34|
|50637c81-fffa-4ee...|              1.0|           1.0|25-34|
|5a023519-f28e-4eb...|              1.0|           1.0|25-34|
|5a781ca

In [301]:
answer2 = answer1.withColumn("gender", decode_gender("prediction_gender"))

In [302]:
answer2.show()

+--------------------+-----------------+--------------+-----+------+
|                 uid|prediction_gender|prediction_age|  age|gender|
+--------------------+-----------------+--------------+-----+------+
|0108d217-e476-493...|              1.0|           1.0|25-34|     M|
|0192cc54-559c-4c8...|              1.0|           1.0|25-34|     M|
|019acd5e-be9a-4cd...|              0.0|           1.0|25-34|     F|
|02e7f830-da57-4d5...|              1.0|           1.0|25-34|     M|
|1d160259-73d8-451...|              1.0|           1.0|25-34|     M|
|1e14a504-276e-448...|              1.0|           1.0|25-34|     M|
|1eb313db-34ff-4bf...|              1.0|           1.0|25-34|     M|
|1eff6e4f-3b8a-447...|              1.0|           1.0|25-34|     M|
|3e75c432-cb78-488...|              1.0|           1.0|25-34|     M|
|47565df3-13e3-460...|              1.0|           1.0|25-34|     M|
|4766a8ab-e9b6-4e0...|              1.0|           1.0|25-34|     M|
|50637c81-fffa-4ee...|            

In [303]:
predictions_df = answer2.select("uid", "gender", "age")

In [304]:
predictions_df.show()

+--------------------+------+-----+
|                 uid|gender|  age|
+--------------------+------+-----+
|0108d217-e476-493...|     M|25-34|
|0192cc54-559c-4c8...|     M|25-34|
|019acd5e-be9a-4cd...|     F|25-34|
|02e7f830-da57-4d5...|     M|25-34|
|1d160259-73d8-451...|     M|25-34|
|1e14a504-276e-448...|     M|25-34|
|1eb313db-34ff-4bf...|     M|25-34|
|1eff6e4f-3b8a-447...|     M|25-34|
|3e75c432-cb78-488...|     M|25-34|
|47565df3-13e3-460...|     M|25-34|
|4766a8ab-e9b6-4e0...|     M|25-34|
|50637c81-fffa-4ee...|     M|25-34|
|5a023519-f28e-4eb...|     M|25-34|
|5a781caa-6131-4d9...|     M|25-34|
|5ab3c7b8-c550-493...|     M|25-34|
|7302e78a-ec04-47e...|     M|25-34|
|73081df3-8f41-435...|     M|25-34|
|89fe85cb-ea4c-4be...|     M|25-34|
|8affa6ce-24c7-4ed...|     M|25-34|
|b2e4450d-c582-441...|     M|25-34|
+--------------------+------+-----+
only showing top 20 rows



In [305]:
predictions_df.count()

5000

In [306]:
kafka_out_df = (
    predictions_df
    .select(f.to_json(f.struct(*predictions_df.columns)).alias('value')).limit(5000)
)

In [307]:
kafka_out_df.show(5, False)

+-------------------------------------------------------------------------+
|value                                                                    |
+-------------------------------------------------------------------------+
|{"uid":"0108d217-e476-493d-8c81-a9744f12451a","gender":"M","age":"25-34"}|
|{"uid":"0192cc54-559c-4c8e-89b4-5f4bf31e4245","gender":"M","age":"25-34"}|
|{"uid":"019acd5e-be9a-4cde-8280-0d2376dc016d","gender":"F","age":"25-34"}|
|{"uid":"02e7f830-da57-4d57-a916-10ff45f5de7b","gender":"M","age":"25-34"}|
|{"uid":"1d160259-73d8-4514-911e-757042b22812","gender":"M","age":"25-34"}|
+-------------------------------------------------------------------------+
only showing top 5 rows



In [308]:
kafka_out_df.count()

5000

In [309]:
kafka_out_df

DataFrame[value: string]

In [310]:
(
    kafka_out_df
    .write
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('topic', KAFKA_OUTPUT_TOPIC)
    .save()
)

In [312]:
kafka_stream = (
    spark
    .readStream
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('subscribe', KAFKA_INPUT_TOPIC)
    .option('startingOffsets', 'earliest')
    .option('failOnDataLoss', 'False')
    .load()
)

In [313]:
kafka_stream.isStreaming

In [ ]:
read_kafka_params = {
    "kafka.bootstrap.servers": KAFKA_BOOTSTRAP_SERVERS,
    "subscribe": INPUT_TOPIC,
    "startingOffsets": "latest"
}
kafka_sdf = spark.readStream.format("kafka").options(**read_kafka_params).load()

In [ ]:
parsed_df = kafka_sdf.select(f.get_json_object(f.col("value").cast("string"), "$.uid").alias("uid"),
                 extract_hosts(f.get_json_object(f.col("value").cast("string"), "$.visits")).alias("hosts"))

In [ ]:
pred_df = model.transform(parsed_df)

In [ ]:
out_df = pred_df.select(f.to_json(f.struct("uid", 
                                           unmap_gender(f.col("prediction")).alias("gender"), 
                                           unmap_age(f.col("prediction")).alias("age"))).alias("value"))

In [314]:
kafka_write_stream = (
    predictions_df
    .select(f.to_json(f.struct(*predictions_df.columns)).alias('value'))
    .writeStream
    .format("kafka")
    .outputMode("append")
    .option("checkpointLocation", "checkpoints/checkpoints_lab04")
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER)
    .option("topic", KAFKA_OUTPUT_TOPIC)
)

In [ ]:
out_df.writeStream.format("kafka").options(**write_kafka_params)\
    .option("checkpointLocation", "streaming/chk/chk_kafka")\
    .outputMode("append").start()

In [ ]:
streams = SparkSession.builder.getOrCreate().streams.active
streams

In [ ]:
if streams:
    for s in streams:
        if s.lastProgress["sources"][0] is not None: 
            desc = s.lastProgress["sources"][0]["description"]
        else:
            desc = "Unknown"
        s.stop()
        print("Stopped {s}".format(s=desc))

In [317]:
spark.stop()

In [318]:
spark

AttributeError: 'NoneType' object has no attribute 'sc'